# Week 2 Notes - Getting and Cleaning Data

## Reading from MySQL
Data in SQL are structured into databases -> databases consist of tables with fields -> tables contain entries as rows. The tables themselves often represent specific aspects of the data which are interlinked within the database - say a table for the salaries of employees, another of the annnual leave, another table for their personal details and so on. 

## Let's install MySQL
In R `install.packages("RMySQL")` 

In julia

In [1]:
using Pkg; Pkg.add("MySQL") ; using MySQL

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
Pkg.add("DataFrames") ; using DataFrames  

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


## Connecting to databases - UCSC genome browser example

Connecting to the UCSC MySQL server and pulling the databases on UCSC that are available to us -- let's do it in R. This will establish a connection to the server - a stream, and then using this stream we will execute a MySQL command `show databases;` to retrieve the available databases, and then disconnect from the stream. 
```R
ucscDB <- dbConnect(MySQL(), user="genome", host="genome-mysql.cse.ucsc.edu")  
result <- dbGetQuery(ucscDB, "show databases;"); dbDisconnect(ucscDB);
```

In Julia we can do this by employing the MySQL.jl package - part of the Databases.jl family;

In [3]:
# Connecting https://mysql.juliadatabases.org/dev/
ucscDB = DBInterface.connect(MySQL.Connection, "genome-mysql.soe.ucsc.edu", "genome")

MySQL.Connection(host="genome-mysql.soe.ucsc.edu", user="genome", port="3306", db="")

In [4]:
# Query the server and store the query in a dataframe - or a csv etc. 
result = DBInterface.execute(ucscDB, "show databases") |> DataFrame; 

In [5]:
# Lets view the result - we can see that it lists all the genomes stored on UCSC
result

Row,Database
,String
1,acaChl1
2,ailMel1
3,allMis1
4,allSin1
5,amaVit1
6,anaPla1
7,ancCey1
8,angJap1
9,anoCar1


In [6]:
"hg38" in result.Database 

true

In [7]:
# Close the connection stream 
DBInterface.close!(ucscDB)

### Now that we've connected to the MySQL server, we will connect to a specific database and perform some queries. 

In R - we'll connect, retrieve all of the table associated with the db, and then execute a funtion to see how many table are stored 
```R
hg38 <- dbConnect(MySQL(), user="genome", db="hg38", host="genome-mysql.soe.ucsc.edu")
allTables <- dbListTables(hg38)
length(allTables)
```

Let's get cracking on Julia

In [8]:
hg38 = DBInterface.connect(MySQL.Connection, "genome-mysql.soe.ucsc.edu", "genome", db="hg38")

MySQL.Connection(host="genome-mysql.soe.ucsc.edu", user="genome", port="3306", db="hg38")

In [9]:
hg38Tables = DBInterface.execute(hg38, "show tables") |> DataFrame; 

In [10]:
hg38Tables[1:10, :]

Row,Tables_in_hg38
,String
1,affyGnf1h
2,affyU133
3,affyU95
4,all_est
5,all_mrna
6,all_sts_primer
7,all_sts_seq
8,altLocations
9,altSeqLiftOverPsl


A whopping 2835 different tables ! UCSC is extreeeeemly information rich, as we can see, there are many many many data sources we can pull from.    

### Now to investigate the specific fields within a specific table

In R; get a table and perform a basic SQL function to show how many fields are in the table
```R
dbListFields(hg38, "all_mrna")
dbGetQuery(hg38, "select count(*) from all_mrna")
```

In Julia

In [11]:
countsql = DBInterface.execute(hg38, "select count(*) from all_mrna") |> DataFrame; 

In [12]:
countsql

Row,count(*)
,Int64
1,10489979


Now lets play with the contents of the fields in R

```R
mrnaData <- dbReadTable(hg38, "all_mrna")
head(mrnaData)
```

Julia !

In [30]:
mrnaData = DBInterface.execute(hg38, "select * from all_mrna;") |> DataFrame

Row,bin,matches,misMatches,repMatches,nCount,qNumInsert,qBaseInsert,tNumInsert,tBaseInsert,strand,qName,qSize,qStart,qEnd,tName,tSize,tStart,tEnd,blockCount,blockSizes,qStarts,tStarts
,UInt16,UInt32,UInt32,UInt32,UInt32,UInt32,UInt32,UInt32,UInt32,String,String,UInt32,UInt32,UInt32,String,UInt32,UInt32,UInt32,UInt32,Array…,Array…,Array…
1,585,1579,25,0,0,0,0,2,884,+,AM992877,1604,0,1604,chr1,248956422,11873,14361,3,"UInt8[0x33, 0x35, 0x34, 0x2c, 0x31, 0x30, 0x39, 0x2c, 0x31, 0x31, 0x34, 0x31, 0x2c]","UInt8[0x30, 0x2c, 0x33, 0x35, 0x34, 0x2c, 0x34, 0x36, 0x33, 0x2c]","UInt8[0x31, 0x31, 0x38, 0x37, 0x33, 0x2c, 0x31, 0x32, 0x36, 0x31, 0x32, 0x2c, 0x31, 0x33, 0x32, 0x32, 0x30, 0x2c]"
2,585,1419,21,0,0,0,0,2,1048,+,AM992881,1440,0,1440,chr1,248956422,11873,14361,3,"UInt8[0x33, 0x35, 0x34, 0x2c, 0x31, 0x32, 0x37, 0x2c, 0x39, 0x35, 0x39, 0x2c]","UInt8[0x30, 0x2c, 0x33, 0x35, 0x34, 0x2c, 0x34, 0x38, 0x31, 0x2c]","UInt8[0x31, 0x31, 0x38, 0x37, 0x33, 0x2c, 0x31, 0x32, 0x35, 0x39, 0x34, 0x2c, 0x31, 0x33, 0x34, 0x30, 0x32, 0x2c]"
3,585,1533,12,0,0,0,0,4,944,+,AM992878,1545,0,1545,chr1,248956422,11873,14362,5,"UInt8[0x33, 0x35, 0x34, 0x2c, 0x35, 0x32, 0x2c, 0x34, 0x33, 0x36, 0x2c, 0x32, 0x39, 0x39, 0x2c, 0x34, 0x30, 0x34, 0x2c]","UInt8[0x30, 0x2c, 0x33, 0x35, 0x34, 0x2c, 0x34, 0x30, 0x36, 0x2c, 0x38, 0x34, 0x32, 0x2c, 0x31, 0x31, 0x34, 0x31, 0x2c]","UInt8[0x31, 0x31, 0x38, 0x37, 0x33, 0x2c, 0x31, 0x32, 0x36, 0x34 … 0x36, 0x35, 0x38, 0x2c, 0x31, 0x33, 0x39, 0x35, 0x38, 0x2c]"
4,585,1578,27,0,0,0,0,2,884,+,AM992879,1605,0,1605,chr1,248956422,11873,14362,3,"UInt8[0x33, 0x35, 0x34, 0x2c, 0x31, 0x30, 0x39, 0x2c, 0x31, 0x31, 0x34, 0x32, 0x2c]","UInt8[0x30, 0x2c, 0x33, 0x35, 0x34, 0x2c, 0x34, 0x36, 0x33, 0x2c]","UInt8[0x31, 0x31, 0x38, 0x37, 0x33, 0x2c, 0x31, 0x32, 0x36, 0x31, 0x32, 0x2c, 0x31, 0x33, 0x32, 0x32, 0x30, 0x2c]"
5,585,1652,0,0,0,0,0,2,884,+,AM992871,1652,0,1652,chr1,248956422,11873,14409,3,"UInt8[0x33, 0x35, 0x34, 0x2c, 0x31, 0x30, 0x39, 0x2c, 0x31, 0x31, 0x38, 0x39, 0x2c]","UInt8[0x30, 0x2c, 0x33, 0x35, 0x34, 0x2c, 0x34, 0x36, 0x33, 0x2c]","UInt8[0x31, 0x31, 0x38, 0x37, 0x33, 0x2c, 0x31, 0x32, 0x36, 0x31, 0x32, 0x2c, 0x31, 0x33, 0x32, 0x32, 0x30, 0x2c]"
6,585,1650,2,0,0,0,0,2,884,+,AM992872,1652,0,1652,chr1,248956422,11873,14409,3,"UInt8[0x33, 0x35, 0x34, 0x2c, 0x31, 0x30, 0x39, 0x2c, 0x31, 0x31, 0x38, 0x39, 0x2c]","UInt8[0x30, 0x2c, 0x33, 0x35, 0x34, 0x2c, 0x34, 0x36, 0x33, 0x2c]","UInt8[0x31, 0x31, 0x38, 0x37, 0x33, 0x2c, 0x31, 0x32, 0x36, 0x31, 0x32, 0x2c, 0x31, 0x33, 0x32, 0x32, 0x30, 0x2c]"
7,585,1648,4,0,0,0,0,2,884,+,AM992875,1652,0,1652,chr1,248956422,11873,14409,3,"UInt8[0x33, 0x35, 0x34, 0x2c, 0x31, 0x30, 0x39, 0x2c, 0x31, 0x31, 0x38, 0x39, 0x2c]","UInt8[0x30, 0x2c, 0x33, 0x35, 0x34, 0x2c, 0x34, 0x36, 0x33, 0x2c]","UInt8[0x31, 0x31, 0x38, 0x37, 0x33, 0x2c, 0x31, 0x32, 0x36, 0x31, 0x32, 0x2c, 0x31, 0x33, 0x32, 0x32, 0x30, 0x2c]"
8,585,1485,3,0,0,0,0,2,1048,+,AM992880,1488,0,1488,chr1,248956422,11873,14409,3,"UInt8[0x33, 0x35, 0x34, 0x2c, 0x31, 0x32, 0x37, 0x2c, 0x31, 0x30, 0x30, 0x37, 0x2c]","UInt8[0x30, 0x2c, 0x33, 0x35, 0x34, 0x2c, 0x34, 0x38, 0x31, 0x2c]","UInt8[0x31, 0x31, 0x38, 0x37, 0x33, 0x2c, 0x31, 0x32, 0x35, 0x39, 0x34, 0x2c, 0x31, 0x33, 0x34, 0x30, 0x32, 0x2c]"
9,585,1631,8,0,0,0,0,4,897,+,BC032353,1673,0,1639,chr1,248956422,11873,14409,5,"UInt8[0x33, 0x35, 0x34, 0x2c, 0x31, 0x30, 0x39, 0x2c, 0x37, 0x33, 0x37, 0x2c, 0x33, 0x30, 0x30, 0x2c, 0x31, 0x33, 0x39, 0x2c]","UInt8[0x30, 0x2c, 0x33, 0x35, 0x34, 0x2c, 0x34, 0x36, 0x33, 0x2c, 0x31, 0x32, 0x30, 0x30, 0x2c, 0x31, 0x35, 0x30, 0x30, 0x2c]","UInt8[0x31, 0x31, 0x38, 0x37, 0x33, 0x2c, 0x31, 0x32, 0x36, 0x31 … 0x39, 0x35, 0x38, 0x2c, 0x31, 0x34, 0x32, 0x37, 0x30, 0x2c]"


In [27]:
DataFrame(mrnaData, mrnaData[:, 1]) 

LoadError: MethodError: no method matching getindex(::MySQL.TextCursor{true}, ::Colon, ::Int64)

In [29]:
Pkg.add("CSV") ; using CSV

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


### Refined queries 
MySQL has a wide range of query options which permit the extration of virtually any aspect of the data, with conditionals, ranges, mismatches and so on. These statements must simply be crafted according to the query structure and provided to the julia functions in order to get what you're looking for.    

For instance we can extract entries from the table which have values in the 'mismatches' column between 1 and 3;   

In R
```R
query <- dbSendQuery(hg38, "select * from all_mrna where misMatches between 1 and 3") 
mrnas <- fetch(query)
```

In Julia

In [32]:
query = DBInterface.execute(hg38, "select * from all_mrna where misMatches between 1 and 3") |> DataFrame

Row,bin,matches,misMatches,repMatches,nCount,qNumInsert,qBaseInsert,tNumInsert,tBaseInsert,strand,qName,qSize,qStart,qEnd,tName,tSize,tStart,tEnd,blockCount,blockSizes,qStarts,tStarts
,UInt16,UInt32,UInt32,UInt32,UInt32,UInt32,UInt32,UInt32,UInt32,String,String,UInt32,UInt32,UInt32,String,UInt32,UInt32,UInt32,UInt32,Array…,Array…,Array…
1,585,1650,2,0,0,0,0,2,884,+,AM992872,1652,0,1652,chr1,248956422,11873,14409,3,"UInt8[0x33, 0x35, 0x34, 0x2c, 0x31, 0x30, 0x39, 0x2c, 0x31, 0x31, 0x38, 0x39, 0x2c]","UInt8[0x30, 0x2c, 0x33, 0x35, 0x34, 0x2c, 0x34, 0x36, 0x33, 0x2c]","UInt8[0x31, 0x31, 0x38, 0x37, 0x33, 0x2c, 0x31, 0x32, 0x36, 0x31, 0x32, 0x2c, 0x31, 0x33, 0x32, 0x32, 0x30, 0x2c]"
2,585,1485,3,0,0,0,0,2,1048,+,AM992880,1488,0,1488,chr1,248956422,11873,14409,3,"UInt8[0x33, 0x35, 0x34, 0x2c, 0x31, 0x32, 0x37, 0x2c, 0x31, 0x30, 0x30, 0x37, 0x2c]","UInt8[0x30, 0x2c, 0x33, 0x35, 0x34, 0x2c, 0x34, 0x38, 0x31, 0x2c]","UInt8[0x31, 0x31, 0x38, 0x37, 0x33, 0x2c, 0x31, 0x32, 0x35, 0x39, 0x34, 0x2c, 0x31, 0x33, 0x34, 0x30, 0x32, 0x2c]"
3,585,925,2,0,0,1,3,7,11787,-,AK310121,930,0,930,chr1,248956422,16630,29344,8,"UInt8[0x31, 0x33, 0x35, 0x2c, 0x31, 0x39, 0x38, 0x2c, 0x31, 0x33 … 0x30, 0x32, 0x2c, 0x31, 0x35, 0x34, 0x2c, 0x32, 0x34, 0x2c]","UInt8[0x30, 0x2c, 0x31, 0x33, 0x35, 0x2c, 0x33, 0x33, 0x33, 0x2c … 0x30, 0x2c, 0x37, 0x35, 0x32, 0x2c, 0x39, 0x30, 0x36, 0x2c]","UInt8[0x31, 0x36, 0x36, 0x33, 0x30, 0x2c, 0x31, 0x36, 0x38, 0x35 … 0x37, 0x33, 0x37, 0x2c, 0x32, 0x39, 0x33, 0x32, 0x30, 0x2c]"
4,585,986,1,0,0,0,0,8,11630,-,AK310139,987,0,987,chr1,248956422,16727,29344,9,"UInt8[0x33, 0x38, 0x2c, 0x31, 0x39, 0x38, 0x2c, 0x31, 0x33, 0x36 … 0x35, 0x38, 0x2c, 0x31, 0x35, 0x34, 0x2c, 0x32, 0x34, 0x2c]","UInt8[0x30, 0x2c, 0x33, 0x38, 0x2c, 0x32, 0x33, 0x36, 0x2c, 0x33 … 0x31, 0x2c, 0x38, 0x30, 0x39, 0x2c, 0x39, 0x36, 0x33, 0x2c]","UInt8[0x31, 0x36, 0x37, 0x32, 0x37, 0x2c, 0x31, 0x36, 0x38, 0x35 … 0x37, 0x33, 0x37, 0x2c, 0x32, 0x39, 0x33, 0x32, 0x30, 0x2c]"
5,585,970,3,0,0,0,0,4,1101,-,AK294377,973,0,973,chr1,248956422,16938,19012,5,"UInt8[0x31, 0x31, 0x37, 0x2c, 0x35, 0x31, 0x30, 0x2c, 0x31, 0x34, 0x37, 0x2c, 0x39, 0x39, 0x2c, 0x31, 0x30, 0x30, 0x2c]","UInt8[0x30, 0x2c, 0x31, 0x31, 0x37, 0x2c, 0x36, 0x32, 0x37, 0x2c, 0x37, 0x37, 0x34, 0x2c, 0x38, 0x37, 0x33, 0x2c]","UInt8[0x31, 0x36, 0x39, 0x33, 0x38, 0x2c, 0x31, 0x37, 0x32, 0x33 … 0x32, 0x36, 0x37, 0x2c, 0x31, 0x38, 0x39, 0x31, 0x32, 0x2c]"
6,585,953,2,0,0,0,0,8,11369,-,AK300161,955,0,955,chr1,248956422,17020,29344,9,"UInt8[0x33, 0x35, 0x2c, 0x31, 0x33, 0x36, 0x2c, 0x31, 0x32, 0x35 … 0x32, 0x37, 0x2c, 0x31, 0x35, 0x34, 0x2c, 0x32, 0x34, 0x2c]","UInt8[0x30, 0x2c, 0x33, 0x35, 0x2c, 0x31, 0x37, 0x31, 0x2c, 0x32 … 0x30, 0x2c, 0x37, 0x37, 0x37, 0x2c, 0x39, 0x33, 0x31, 0x2c]","UInt8[0x31, 0x37, 0x30, 0x32, 0x30, 0x2c, 0x31, 0x37, 0x32, 0x33 … 0x37, 0x33, 0x37, 0x2c, 0x32, 0x39, 0x33, 0x32, 0x30, 0x2c]"
7,585,775,1,0,0,0,0,6,10897,-,AK308540,776,0,776,chr1,248956422,17671,29344,7,"UInt8[0x37, 0x31, 0x2c, 0x31, 0x34, 0x37, 0x2c, 0x39, 0x35, 0x2c … 0x32, 0x37, 0x2c, 0x31, 0x35, 0x34, 0x2c, 0x32, 0x34, 0x2c]","UInt8[0x30, 0x2c, 0x37, 0x31, 0x2c, 0x32, 0x31, 0x38, 0x2c, 0x33 … 0x31, 0x2c, 0x35, 0x39, 0x38, 0x2c, 0x37, 0x35, 0x32, 0x2c]","UInt8[0x31, 0x37, 0x36, 0x37, 0x31, 0x2c, 0x31, 0x37, 0x39, 0x31 … 0x37, 0x33, 0x37, 0x2c, 0x32, 0x39, 0x33, 0x32, 0x30, 0x2c]"
8,585,974,2,0,0,0,0,0,0,-,AK311358,976,0,976,chr1,248956422,29043,30019,1,"UInt8[0x39, 0x37, 0x36, 0x2c]","UInt8[0x30, 0x2c]","UInt8[0x32, 0x39, 0x30, 0x34, 0x33, 0x2c]"
9,585,1123,1,0,0,0,0,2,341,-,AY341950,1124,0,1124,chr1,248956422,34612,36077,3,"UInt8[0x35, 0x36, 0x32, 0x2c, 0x32, 0x30, 0x35, 0x2c, 0x33, 0x35, 0x37, 0x2c]","UInt8[0x30, 0x2c, 0x35, 0x36, 0x32, 0x2c, 0x37, 0x36, 0x37, 0x2c]","UInt8[0x33, 0x34, 0x36, 0x31, 0x32, 0x2c, 0x33, 0x35, 0x32, 0x37, 0x36, 0x2c, 0x33, 0x35, 0x37, 0x32, 0x30, 0x2c]"


In [34]:
first(query)

Row,bin,matches,misMatches,repMatches,nCount,qNumInsert,qBaseInsert,tNumInsert,tBaseInsert,strand,qName,qSize,qStart,qEnd,tName,tSize,tStart,tEnd,blockCount,blockSizes,qStarts,tStarts
,UInt16,UInt32,UInt32,UInt32,UInt32,UInt32,UInt32,UInt32,UInt32,String,String,UInt32,UInt32,UInt32,String,UInt32,UInt32,UInt32,UInt32,Array…,Array…,Array…
1,585,1650,2,0,0,0,0,2,884,+,AM992872,1652,0,1652,chr1,248956422,11873,14409,3,"UInt8[0x33, 0x35, 0x34, 0x2c, 0x31, 0x30, 0x39, 0x2c, 0x31, 0x31, 0x38, 0x39, 0x2c]","UInt8[0x30, 0x2c, 0x33, 0x35, 0x34, 0x2c, 0x34, 0x36, 0x33, 0x2c]","UInt8[0x31, 0x31, 0x38, 0x37, 0x33, 0x2c, 0x31, 0x32, 0x36, 0x31, 0x32, 0x2c, 0x31, 0x33, 0x32, 0x32, 0x30, 0x2c]"


In [ ]:
size(query)

### Close the connection!

In [37]:
DBInterface.close!(hg38)

MySQL.Connection(disconnected)